In [7]:
from glob import glob
import pandas as pd

In [8]:
columns = ['price', 'quantity', 'timestamp', 'is_buyer_maker']

spot_df = []
derivative_df = []

for file in glob('data/*'):
    if 'spot' in file:
        spot_df.append(pd.read_csv(file, header=None))
    else:
        derivative_df.append(pd.read_csv(file, header=None)[1:])

spot_df = pd.concat(spot_df)[[1, 2, 5, 6]].reset_index(drop=True)
derivative_df = pd.concat(derivative_df)[[1, 2, 5, 6]].reset_index(drop=True)

spot_df.columns = columns
derivative_df.columns = columns



spot_df['timestamp'] = pd.to_datetime(spot_df['timestamp'], unit='ms')
derivative_df['timestamp'] = pd.to_datetime(derivative_df['timestamp'], unit='ms')


spot_df = spot_df.sort_values('timestamp').reset_index(drop=True)
derivative_df = derivative_df.sort_values('timestamp').reset_index(drop=True)


derivative_df['derivative_price'] = derivative_df['price']


/tmp/ipykernel_82350/1821862887.py:10: DtypeWarning: Columns (0,1,2,3,4,5,6) have mixed types. Specify dtype option on import or set low_memory=False.
  derivative_df.append(pd.read_csv(file, header=None)[1:])
/tmp/ipykernel_82350/1821862887.py:10: DtypeWarning: Columns (0,1,2,3,4,5,6) have mixed types. Specify dtype option on import or set low_memory=False.
  derivative_df.append(pd.read_csv(file, header=None)[1:])
/tmp/ipykernel_82350/1821862887.py:10: DtypeWarning: Columns (0,1,2,3,4,5,6) have mixed types. Specify dtype option on import or set low_memory=False.
  derivative_df.append(pd.read_csv(file, header=None)[1:])


In [9]:
df = pd.merge_asof(spot_df[['timestamp', 'price', 'quantity', 'is_buyer_maker']], derivative_df[['timestamp', 'derivative_price']], on='timestamp')

In [17]:
df = df[:30000]

In [18]:
df

,timestamp,price,quantity,is_buyer_maker,derivative_price
0,2023-02-01 00:00:00.000,23125.13,0.00146,True,NaN
1,2023-02-01 00:00:00.005,23126.42,0.00340,False,NaN
2,2023-02-01 00:00:00.008,23126.42,0.00168,False,NaN
3,2023-02-01 00:00:00.010,23126.42,0.00090,False,NaN
4,2023-02-01 00:00:00.010,23126.42,0.00123,False,NaN
...,...,...,...,...,...
29995,2023-02-01 00:05:43.910,23103.42,0.04000,True,23095.60
29996,2023-02-01 00:05:43.910,23103.37,0.00558,True,23095.60
29997,2023-02-01 00:05:44.108,23103.33,0.00352,True,23096.50
29998,2023-02-01 00:05:44.108,23103.36,0.04000,True,23096.50


In [19]:
params = {}
params['capital'] = 20000
params['maxPositionSize'] = 10000

curr_pos = 0
hedge_pos = 0

buys = 0
sells = 0
buy_volume = 0
sell_volume = 0

deets = []


for idx, row in df.iterrows():
    curr_deets = {}

    if (idx % 100) == 0:
        curr_deets['timestamp'] = row['timestamp']
        curr_deets['curr_pos'] = curr_pos * row['price']
        curr_deets['capital'] = params['capital']
        curr_deets['worth'] = curr_deets['curr_pos'] + curr_deets['capital']
        curr_deets['price'] = row['price']
        deets.append(curr_deets)
        
    if row['is_buyer_maker']:
        if (curr_pos * row['price']) < params['maxPositionSize']:

            buy_amt = row['quantity'] * row['price']
            quantity = row['quantity']

            if buy_amt > params['maxPositionSize']:
                buy_amt = params['maxPositionSize']
                quantity = buy_amt / row['price']

            buys = buys + 1
            buy_volume = buy_volume + buy_amt

            # print("BUYING {} BTC for {} USD. Current POS: {}".format(quantity, buy_amt, curr_pos))
            curr_pos = curr_pos + quantity
            params['capital'] = params['capital'] - buy_amt
    else:
        if curr_pos > 0:
            sell_amt = row['quantity'] * row['price']
            quantity = row['quantity']

            if sell_amt > curr_pos:
                sell_amt = curr_pos
                quantity = sell_amt / row['price']

            sells = sells + 1
            sell_volume = sell_volume + sell_amt

            # print("SELLING {} BTC for {} USD. Current POS: {}".format(quantity, sell_amt, curr_pos))
            curr_pos = curr_pos - quantity
            params['capital'] = params['capital'] + sell_amt

In [20]:
print(buys, sells, buy_volume, sell_volume)

30 14972 19778.868232200002 7558.665062570765


In [21]:
details_df = pd.DataFrame(deets)

In [22]:
import plotly.graph_objects as go

fig = go.Figure(layout=go.Layout(xaxis={'spikemode': 'across'}))
fig.add_trace(go.Scatter(x=details_df['timestamp'], y=details_df['curr_pos'], name='curr_pos', yaxis="y1"))
fig.add_trace(go.Scatter(x=details_df['timestamp'], y=details_df['worth'], name='worth', yaxis="y2"))
fig.add_trace(go.Scatter(x=details_df['timestamp'], y=details_df['price'], name='price', yaxis="y3"))

fig.update_layout(
            yaxis=dict(
                titlefont=dict(
                    color="#000000"
                ),
                tickfont=dict(
                    color="#000000"
                )
            ),
            yaxis2=dict(
                anchor="free",
                overlaying="y",
                side="left",
                position=1
            ),
            yaxis3=dict(
                anchor="free",
                overlaying="y",
                side="right",
                position=1
            )
            )

In [23]:
open('plot_short.html', 'w').write(fig.to_html())

3622957